# Label side

In [2]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-04-10"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2773440, 6)

In [3]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2773440, 6)

In [2]:
from custom_indicators.toolbox.bar.dollar_bar import estimate_dollar_bar_threshold

for i in [200, 600, 1800]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

200m: 1784228506.0705855
600m: 5352685516.211758
1800m: 16177005113.639977


In [3]:
from jesse import helpers

from custom_indicators.toolbox.bar.dollar_bar import build_dollar_bar
from strategies.BinanceMLV2.config import DOLLAR_BAR_THRESHOLD_MID

print(DOLLAR_BAR_THRESHOLD_MID)

tp = helpers.date_to_timestamp("2025-04-05")

label_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)

print(label_bar.shape)

5352685516.211758
(4642, 6)


In [4]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(label_bar[:, 2], 0.05)
np.unique(res, return_counts=True)

(array([-1,  1]), array([2117, 2525]))

In [5]:
label = np.column_stack((label_bar, res))
label

array([[ 1.57799880e+12,  6.94495000e+03,  7.84890000e+03, ...,
         6.86344000e+03,  7.17060163e+05, -1.00000000e+00],
       [ 1.57840344e+12,  7.84861000e+03,  7.81022000e+03, ...,
         7.73300000e+03,  6.62080816e+05, -1.00000000e+00],
       [ 1.57861170e+12,  7.81023000e+03,  8.45870000e+03, ...,
         7.67000000e+03,  6.63591450e+05,  1.00000000e+00],
       ...,
       [ 1.74421926e+12,  7.91491000e+04,  8.15480000e+04, ...,
         7.91491000e+04,  7.02868230e+04,  1.00000000e+00],
       [ 1.74422094e+12,  8.15493000e+04,  8.20514000e+04, ...,
         8.11917000e+04,  6.55216160e+04,  1.00000000e+00],
       [ 1.74422592e+12,  8.20523000e+04,  8.25880000e+04, ...,
         8.19801000e+04,  6.48128060e+04,  1.00000000e+00]])

In [6]:
np.save("data/label_side.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(4642, 7)
(4614, 7)


In [7]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    label_bar[label_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[label_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-02 21:00:00,2020-01-02 21:00:00,6944.95,7848.9,8014.91,6863.44,717060.163,-1


In [8]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from strategies.BinanceMLV2.config import SIDE_ALL

label_side_1h = np.load("data/label_side.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_600m.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="strategies/BinanceMLV2/model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(4614, 7)
(array([-1.,  1.]), array([2100, 2514]))
(4614, 15051)
(array([-1,  1]), array([2078, 2536]))


In [2]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([ 802, 3812]))

In [3]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta.npy", label_meta_1h)